# 패키지 선언

In [1]:
# 패키지 불러오기

import requests
import bs4
from bs4 import BeautifulSoup
import os
import random
import hashlib
import urllib.request
import urllib
from urllib.parse import urlparse, urlencode, parse_qs
import pandas as pd
import csv
import re

from datetime import datetime, timedelta
from urllib import parse
import time
import requests
import numpy as np
import json
from random import *
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent

from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException

ua = UserAgent()
userAgent = ua.random
print(userAgent)

options = Options()
options.add_argument(f"user-agent={userAgent}")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--proxy-server=socks5://127.0.0.1:9050')

executable_path = '/home/book_crawling/geckodriver' # chrome driver의 경로
driver = webdriver.Firefox(executable_path=executable_path, options=options)

Mozilla/5.0 (Windows; U; Windows NT 6.1; tr-TR) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27


# 반디앤루니스 크롤링 함수

In [2]:
def parse_bandi(isbn):
    
    book = {'isbn' : isbn,
            '제목' : '없음',
            '저자' : '없음',
            '역자' : '없음',
            '출판일' : '없음',
            '출판사' : '없음',
            '별점' : '없음',
            '리뷰' : '없음',
            '카테고리' : '없음',
            '책소개' : '없음',
            '저자소개' : '없음',
            '추천사' : '없음',
            '목차' : '없음',
            '책속으로' : '없음',
            '출판사리뷰' : '없음'
           }
    
    front_page = 'http://www.bandinlunis.com/search/search.do?q='
    driver.get(front_page + isbn)
    time.sleep(0.5)
    
    try:
        book_contents = driver.find_element_by_class_name('book_contents')
        title_link = book_contents.find_element_by_tag_name('strong')
        title_link.click()
        link_more = driver.find_elements_by_class_name('link_more')
        for l in link_more:
            l.click()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    except:
        return book

    # 제목
    try:
        title_el = soup.select_one('.group_title')
        main = title_el.select_one('.txt_main')
        try:
            sub = title_el.select_one('.txt_sub')
            title = main.text.strip() + ' - ' + sub.text.strip()
        except:
            title = main.text.strip()
    except:
        title = '없음'
    book['제목'] = title
    
    # 저자, 역자, 출판일 - soup
    try:
        group_inside = soup.select_one('.group_inside')
        lis = group_inside.select('li')

        author = '없음'
        translator = '없음'
        publisher = '없음'
        pub_date = pub_date = lis[-1].text
        for li in lis:
            if '저' in li.text:
                author = li.text.replace(' 저', '')
            if '역' in li.text:
                translator = li.text.replace(' 역', '')
            if 'Publish' in li.a['href']:
                publisher = li.text
        pub_date = lis[-1].text
    except:
        pass
    book['저자'] = author
    book['역자'] = translator
    book['출판일'] = pub_date
    book['출판사'] = publisher
            
    # 별점
    try:
        rating_el = soup.select_one('.medium_ratings_num')
        rating = re.search('\d.{1,4}\\n', rating_el.text).group().replace('\n', '')
    except:
        rating = '없음'
    book['별점'] = rating
    
    # 리뷰
    try: 
        review_el = rating_el.select_one('.ml10')
        review_num = review_el.strong.text + ' (반디앤루니스)'
    except:
        review_num = '없음'
    book['리뷰'] = review_num
    
    # 카테고리
    try:
        categories = soup.find_all('li', class_='mb10')
        category = [x.text.replace('\n', '').replace('\t', '') for x in categories]
    except:
        category = []
    book['카테고리'] = category
    
    # 책소개, 추천사
    try:
        shot_desc = soup.select_one('#cutBookDesc')
        desc = shot_desc.text.replace('\t', '')
        book_intro = desc.split('추천글')[0]
        recommend = desc.split('추천글')[1]
    except:
        book_intro = '없음'
        recommend = '없음'
    book['책소개'] = book_intro
    book['추천사'] = recommend
    
    # 저자소개
    try:
        author_name = soup.select_one('.author_name').text.strip()
        profile = soup.select_one('.txt_profile_marge').text.strip()
        author_intro = author_name + '\n' + profile
    except:
        author_intro = '없음'
    book['저자소개'] = author_intro
    
    # 목차
    try:
        table_title = soup.find('h3', class_='txt_title', string=re.compile('목차'))
        table_el = str(table_title.parent.next_sibling.next_sibling)
        table = re.sub('\<[^>]*\>', '\n', table_el).replace('\t', '').replace('닫기', '').strip()
    except:
        table = '없음'
    book['목차'] = table
    
    return book

# csv파일 불러오기 & 크롤링

In [3]:
book_specs = pd.read_csv('./book_spec_korean_isbns.csv', low_memory=False, index_col=0)
isbns = book_specs['0'].tolist()
len(isbns)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


2872189

In [4]:
sample_isbn = '9788947547079'
parse_bandi(sample_isbn)

{'isbn': '9788947547079',
 '제목': '싱크 어게인 - 모르는 것을 아는 힘',
 '저자': '애덤 그랜트',
 '역자': '이경식 ',
 '출판일': '2021년 03월 30일',
 '출판사': '한국경제신문',
 '별점': '9.8',
 '리뷰': '5 (반디앤루니스)',
 '카테고리': ['경제/경영  > 경영  > 경영일반  > 경영일반', '자기계발  > 비즈니스능력계발  > 비즈니스소양'],
 '책소개': '\n출판사 리뷰\n세계적인 베스트셀러 《오리지널스》 애덤 그랜트의 5년 만의 신작“다시 생각하기가 지금보다 더 필요한 때는 없었다!”★★★출간 즉시 아마존 베스트셀러 / 빌 앤 멀린다 게이츠 강력 추천 / 전 세계 32개국 출간★★★불확실성으로 가득한 세상 ‘모르는 것을 아는 능력’도 지능의 일부이다 확증 편향, 소망 편향 등 스스로를 무너뜨리는 ‘확신의 함정’에서 벗어나는 법와튼스쿨 최연소 종신교수이자 도발적 경영 사상가인 애덤 그랜트가 ‘다시 생각하기’라는 매주 중요한 사고법에 대해 고찰한다. 즉, 타인의 의견을 묻고, 마음을 여는 법을 배우는 것이다. 이는 일과 삶에 있어 탁월하고 지혜로운 위치에 설 수 있게 한다. 지능은 보통 생각하고 배우는 능력으로 보이지만, 급변하는 세계에서는 더 중요한 또 다른 세트의 인지 능력이 있다. 그것은 “다시 생각하고 배운 것을 고의적으로 잊는” 능력이다.우리는 일상생활에서, 너무 많은 사람들이 ‘의심의 불편함’보다 ‘확신의 편안함’을 선호한다. 스스로를 힘들게 하는 생각 대신 기분 좋게 만드는 의견을 듣게 되는 것이다. ‘의견 불일치’를 ‘배울 수 있는 기회’라기보다는 ‘자아에 대한 위협’으로 본다. 그래서 우리는 우리의 사고 과정에 도전하는 사람들에게 끌려야 할 때, 우리의 결론에 동의하는 사람들로 둘러싸여 있게 된다.애덤 그랜트는 <뉴욕타임스> 1위의 베스트셀러 《오리지널스》와 《기브앤테이크》의 작가로서, 그는 자신이 옳다고 여김에도, 자신이 틀렸다는 듯이 경청하는 것을 인생의 지

# 대량 크롤링

In [ ]:
num = 1680000
count_save = num - 10000

results = []
for isbn in tqdm(isbns[num - 10000:num]):
    time.sleep(1)
    try:
        result = parse_bandi(str(isbn))
        results.append(result)
    except:
        result = {'isbn' : str(isbn),
                  '제목' : '없음',
                  '저자' : '없음',
                  '역자' : '없음',
                  '출판일' : '없음',
                  '출판사' : '없음',
                  '별점' : '없음',
                  '리뷰' : '없음',
                  '카테고리' : '없음',
                  '책소개' : '없음',
                  '저자소개' : '없음',
                  '추천사' : '없음',
                  '목차' : '없음',
                  '책속으로' : '없음',
                  '출판사리뷰' : '없음'
                 }
        results.append(result)
        
    count_save += 1
    
    if count_save % 1000 == 0:
        file_name = f'bandi_{count_save}_2,870,000.csv'
        results_df = pd.DataFrame(results)
        results_df.to_csv('./crawled_this_week/' + file_name, encoding='utf-8-sig')
        results = []
        print(f'saving {count_save}th results to {file_name}')
        
print('Crawling Completed')

  0%|          | 0/10000 [00:00<?, ?it/s]